### Neural Network Creation and Training:

The following code creates a Neural Network from scratch, and trains the Neural Network based on the MNIST dataset

In [ ]:
import torch
import torch.nn as nn
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

import numpy as np

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else "cpu"

In [ ]:
# Structure adopted partially from the PyTorch Documentation page, basic Neural Network:

# Note: The input images from MNIST will be in 28x28 dimensions, so linear input should be in 784:

class MNISTNeuralNetwork(nn.Module):
    def __init__(self, input_size, output_size):
      super().__init__()
      self.flatten = nn.Flatten()
      self.linear_relu_stack = nn.Sequential(
          nn.Linear(input_size, 256),
          nn.ReLU(),
          nn.Linear(256, 256),
          nn.ReLU(),
          nn.Linear(256, output_size),
      )

    def forward(self, x):
      x = self.flatten(x)
      logits = self.linear_relu_stack(x)
      return logits

### Loading the MNIST Data:

In order to load the MNIST Data into the notebook, we will need to make use of datasets and DataLoader modules as provided in PyTorch:

In [ ]:
training_dataset = datasets.MNIST(root='data/', train=True, download=True, transform=transforms.ToTensor())
training_loader = DataLoader(training_dataset, batch_size=64)

test_dataset = datasets.MNIST(root='data/', train=False, download=True, transform=transforms.ToTensor())
test_loader = DataLoader(test_dataset, batch_size=64)

In [ ]:
# Initialize the Network:
model = MNISTNeuralNetwork(input_size = 784, output_size=10).to(device)

# Declare Loss Function and Optimizer:
loss_fcn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)
optimizer.zero_grad()

### Training the Actual Network:

For this project we will train the model on CLEAN data, but will test the neural network on noisy data. For this reason, we will need to alter the test dataset images later on in the notebook. However, for now - we will train the neural network that we have created over 100 epochs.

In [ ]:
epochs = 100
past_training_loss = np.inf

for epoch in range(epochs):

  if epoch != 1:
    past_training_loss = training_loss

  training_loss = 0

  for idx, (data, label) in enumerate(training_loader):
    data = data.to(device)
    labels = label.to(device)

    data_flat = data.reshape(-1, 784)

    # Make the prediction based on the NN model:
    scores = model(data_flat)

    # Compute the loss value based on the pre-defined loss function:
    loss_val = loss_fcn(scores, labels)

    # Backpropagation, allow for:
    optimizer.zero_grad()
    loss_val.backward()
    optimizer.step()

    training_loss += loss_val.item() * data.size(0)

  print(f"Epoch Number: {epoch}, Training Loss: {training_loss}")
  epoch += 1

### Testing Accuracy and Loss:

Now we will test the model on the testing data, and assess the loss values and accuracy:

In [ ]:
total_correct = 0
total_counter = 0

test_loss = 0

for data, labels in test_loader:

  data = data.to(device)
  labels = labels.to(device)

  test_data_flat = data.reshape(-1, 784)

  predictions = model(test_data_flat)

  for pred, label in zip(predictions, labels):
    if torch.argmax(pred).item() == label.item():
      total_correct += 1
    total_counter += 1

  loss = loss_fcn(predictions, labels)
  test_loss += loss.item() * data.size(0)

print(f"Accuracy: {(total_correct/total_counter) * 100}")
print(f"Test Loss: {test_loss/len(test_loader)}")

In [ ]:
 # Save the model:
torch.save(model.state_dict(), 'trained_model.pth')

### Adding Noise to the Test Dataset:

One of the main purposes of this project is to see whether the Neural Network is robust enough to handle noise in a dataset. Given that the Neural Network is trained on a clean dataset, we want to see if adding noise from other distributions would make a huge difference in Neural Network classification accuracy.